___

<a
align='right' /></a>
#### Data Collection: Venue Data
___
<h3 align="Left">by Mohammd Aboutorabi,Fatemeh Rahideh,Mohammad Yousofvand
 </h3> 
 
___



## Collecting Venue Data using Foursquare's API

---
**GOOD NEWS!** To use the recommender system, you don't need to rerun and scrap the web. I've made the data available for you **<a href='https://github.com/eliasmelul/finding_schitts/blob/master/Data/one_hot_encoded_usa_FS.csv'>here</a>**.

Want direct access? **<a href='https://s3.us-east-2.amazonaws.com/www.findingmyschittscreek.com/Data/one_hot_encoded_usa_FS.csv'>Click Here</a>**

---
This data is a one-hot encoded dataframe of what category of venues are in the top 100 in every city.

----


In [1]:
import pandas as pd # import pandas for dataframes
import numpy as np
import requests
from bs4 import BeautifulSoup
import locale
from datetime import datetime
import re
import matplotlib.pyplot as plt
import seaborn as sns
import json
import folium

import matplotlib.cm as cm
import matplotlib.colors as colors

from IPython.display import HTML, display
from IPython.display import Image 
from IPython.core.display import HTML 

pd.options.display.max_columns = None
pd.options.display.max_rows=None


%matplotlib inline

In [2]:
scraped_data = pd.read_csv('https://s3.us-east-2.amazonaws.com/www.findingmyschittscreek.com/Data/scraped_datausa.csv', index_col=0)
scraped_data.head()

City  Population  Population Change  Poverty Rate  Median Age  \
0   Los Angeles, CA  13300000.0           -0.46700         15.80        37.1   
1        Dallas, TX   7540000.0            1.89000         13.30        35.1   
2    Washington, DC   6250000.0            0.55500          8.28        37.1   
3  Philadelphia, PA   6100000.0            0.00413         13.10        38.8   
4     Riverside, CA   4620000.0            0.91000         16.80        34.8   

   Median Household Income  Median Household Income Change  Number Employees  \
0                  72563.0                            3.67         6620000.0   
1                  69445.0                            3.06         3820000.0   
2                 102180.0                            2.52         3370000.0   
3                  70747.0                            3.17         3020000.0   
4                  65671.0                            5.93         2020000.0   

   Number Employees Change  Median Property Value  \
0                    0.499               650300.0   
1                    2.890               238800.0   
2                    0.729               433200.0   
3                    1.640               256400.0   
4                    3.810               360300.0   

   Median Property Value Change  Average Male Salary  Average Female Salary  \
0                          5.38              78575.0                62298.0   
1                         11.10              67436.0                48835.0   
2                          2.03              77546.0                58156.0   
3                          2.19              69779.0                52119.0   
4                          5.26              78575.0                62298.0   

   Gender Salary Ratio M2F  Gini 2018  Gini Change  \
0                 1.261276      0.499     0.402414   
1                 1.380895      0.479     0.000000   
2                 1.333414      0.480     0.628931   
3                 1.338840      0.469    -0.635593   
4                 1.261276      0.499     0.402414   

   Patient to Clinician Ratio  Foreign Born Population Ratio  \
0                      1281.0                           33.2   
1                      1670.0                           17.4   
2                      1316.0                           22.2   
3                      1232.0                           10.5   
4                      1281.0                           20.9   

   Citizens Percentage  Total Degrees  Degrees Ratio M2F  Degrees per Capita  \
0                 84.7       247648.0           0.724208            0.018620   
1                 88.2        84389.0           0.676980            0.011192   
2                 89.5       102309.0           0.864843            0.016369   
3                 95.0       100077.0           0.723712            0.016406   
4                 89.9        44650.0           0.673476            0.009665   

   Households  People Per House  Homeownership  Commute Time  
0   4350000.0          3.057471           48.2          29.4  
1   2620000.0          2.877863           59.1          26.8  
2   2230000.0          2.802691           63.5          32.8  
3   2290000.0          2.663755           66.6          28.8  
4   1370000.0          3.372263           64.6          31.1

**Collecting and Adding location data**

To use Foursquare's API, we must first add a latitude and longitude attribute to all the cities possible.

In [3]:
# list_cities = list(scraped_data.City)
# list_cities = [i+", USA" for i in list_cities]

# from geopy.geocoders import Nominatim
# geolocator = Nominatim(user_agent="new_fkf_finding_my_shitts_creek")
# locations_list = []

# for i in list_cities:
#     try:
#         location = geolocator.geocode(i, timeout=10)
#         city = i[:-5]
#         lat = location.latitude
#         long = location.longitude
#         dic_lats = {"City":city,"Latitude":lat,"Longitude":long}
#         locations_list.append(dic_lats)
#         print(dic_lats)
#     except:
#         try:
#             geolocator = Nominatim(user_agent="wowzadayum_lol")
#             location = geolocator.geocode(i, timeout=10)
#             city = i[:-5]
#             lat = location.latitude
#             long = location.longitude
#             dic_lats = {"City":city,"Latitude":lat,"Longitude":long}
#             locations_list.append(dic_lats)
#         except:
#             try:
#                 geolocator = Nominatim(user_agent="hello_its_me_FMSC")
#                 location = geolocator.geocode(i, timeout=10)
#                 city = i[:-5]
#                 lat = location.latitude
#                 long = location.longitude
#                 dic_lats = {"City":city,"Latitude":lat,"Longitude":long}
#                 locations_list.append(dic_lats)
#             except:
#                 pass


I first tried getting the data using Geopy. Unfortunately, it was too volatile and unreliable and had to look for other solutions.

I found <a href='https://simplemaps.com/data/us-cities'>this</a> website from which you can download the data. 

In [4]:
cities_latlong = pd.read_csv('https://s3.us-east-2.amazonaws.com/www.findingmyschittscreek.com/Data/uscities.csv')
cities_latlong.head()

city    city_ascii state_id  state_name  county_fips county_name  \
0   South Creek   South Creek       WA  Washington        53053      Pierce   
1        Roslyn        Roslyn       WA  Washington        53037    Kittitas   
2       Sprague       Sprague       WA  Washington        53043     Lincoln   
3    Gig Harbor    Gig Harbor       WA  Washington        53053      Pierce   
4  Lake Cassidy  Lake Cassidy       WA  Washington        53061   Snohomish   

  county_fips_all county_name_all      lat       lng  population  density  \
0           53053          Pierce  46.9994 -122.3921      2500.0    125.0   
1           53037        Kittitas  47.2507 -121.0989       947.0     84.0   
2           53043         Lincoln  47.3048 -117.9713       441.0    163.0   
3           53053          Pierce  47.3352 -122.5968      9507.0    622.0   
4           53061       Snohomish  48.0639 -122.0920      3591.0    131.0   

    source  military  incorporated             timezone  ranking  \
0  polygon     False          True  America/Los_Angeles        3   
1  polygon     False          True  America/Los_Angeles        3   
2  polygon     False          True  America/Los_Angeles        3   
3  polygon     False          True  America/Los_Angeles        3   
4  polygon     False          True  America/Los_Angeles        3   

                zips          id  
0  98580 98387 98338  1840042075  
1  98941 98068 98925  1840019842  
2              99032  1840021107  
3        98332 98335  1840019855  
4  98223 98258 98270  1840041959

To use this dataset and begin scraping venue information for each city, all we need to do is create a city variable that includes the state, and combine the general data and the location dataset.

In [5]:
cities_latlong['City'] = [row.city+", "+row.state_id for i, row in cities_latlong.iterrows()]
cities_latlong.City[0:5]

0     South Creek, WA
1          Roslyn, WA
2         Sprague, WA
3      Gig Harbor, WA
4    Lake Cassidy, WA
Name: City, dtype: object

In [6]:
cities_loc_all = scraped_data.merge(cities_latlong, how='inner', on="City")
print(cities_loc_all.shape)
cities_loc_all.head()

(3964, 45)


City  Population  Population Change  Poverty Rate  Median Age  \
0   Los Angeles, CA  13300000.0           -0.46700         15.80        37.1   
1        Dallas, TX   7540000.0            1.89000         13.30        35.1   
2    Washington, DC   6250000.0            0.55500          8.28        37.1   
3  Philadelphia, PA   6100000.0            0.00413         13.10        38.8   
4     Riverside, CA   4620000.0            0.91000         16.80        34.8   

   Median Household Income  Median Household Income Change  Number Employees  \
0                  72563.0                            3.67         6620000.0   
1                  69445.0                            3.06         3820000.0   
2                 102180.0                            2.52         3370000.0   
3                  70747.0                            3.17         3020000.0   
4                  65671.0                            5.93         2020000.0   

   Number Employees Change  Median Property Value  \
0                    0.499               650300.0   
1                    2.890               238800.0   
2                    0.729               433200.0   
3                    1.640               256400.0   
4                    3.810               360300.0   

   Median Property Value Change  Average Male Salary  Average Female Salary  \
0                          5.38              78575.0                62298.0   
1                         11.10              67436.0                48835.0   
2                          2.03              77546.0                58156.0   
3                          2.19              69779.0                52119.0   
4                          5.26              78575.0                62298.0   

   Gender Salary Ratio M2F  Gini 2018  Gini Change  \
0                 1.261276      0.499     0.402414   
1                 1.380895      0.479     0.000000   
2                 1.333414      0.480     0.628931   
3                 1.338840      0.469    -0.635593   
4                 1.261276      0.499     0.402414   

   Patient to Clinician Ratio  Foreign Born Population Ratio  \
0                      1281.0                           33.2   
1                      1670.0                           17.4   
2                      1316.0                           22.2   
3                      1232.0                           10.5   
4                      1281.0                           20.9   

   Citizens Percentage  Total Degrees  Degrees Ratio M2F  Degrees per Capita  \
0                 84.7       247648.0           0.724208            0.018620   
1                 88.2        84389.0           0.676980            0.011192   
2                 89.5       102309.0           0.864843            0.016369   
3                 95.0       100077.0           0.723712            0.016406   
4                 89.9        44650.0           0.673476            0.009665   

   Households  People Per House  Homeownership  Commute Time          city  \
0   4350000.0          3.057471           48.2          29.4   Los Angeles   
1   2620000.0          2.877863           59.1          26.8        Dallas   
2   2230000.0          2.802691           63.5          32.8    Washington   
3   2290000.0          2.663755           66.6          28.8  Philadelphia   
4   1370000.0          3.372263           64.6          31.1     Riverside   

     city_ascii state_id            state_name  county_fips  \
0   Los Angeles       CA            California         6037   
1        Dallas       TX                 Texas        48113   
2    Washington       DC  District of Columbia        11001   
3  Philadelphia       PA          Pennsylvania        42101   
4     Riverside       CA            California         6065   

            county_name                county_fips_all  \
0           Los Angeles                          06037   
1                Dallas  48113|48085|48121|48257|48397   
2  District of Columbia                       

Awesome! We may have lost a few cities in merging datasets, but it's acceptable for our overall analysis. With about 4000 cities and their profiles, the analysis will be quite comprehensible.

---

Now that we have the cities and their locations, lets begin using the Foursquare API.

---

**Exploring Venue Information from Foursquare**

In [7]:
CLIENT_ID = '###' # Your Foursquare ID
CLIENT_SECRET = '###' # Your Foursquare Secret
VERSION = '20180604'

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: ###
CLIENT_SECRET:###


In [8]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [16]:
LIMIT=100
def get_Nearby_Venues(names, latitudes, longitudes, radius=5000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
                    
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
        
        print(requests.get(url).json())
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['City', 
                  'City Latitude', 
                  'City Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [17]:
try: 
    usa_venues = get_Nearby_Venues(names=cities_loc_all.City, latitudes=cities_loc_all.lat, longitudes=cities_loc_all.lng)
except:
    try:
        usa_venues = get_Nearby_Venues(names=cities_loc_all.City, latitudes=cities_loc_all.lat, longitudes=cities_loc_all.lng)
    except:
        usa_venues = get_Nearby_Venues(names=cities_loc_all.City, latitudes=cities_loc_all.lat, longitudes=cities_loc_all.lng)
        
print(usa_venues.shape)
usa_venues.head()

{'meta': {'code': 400, 'errorType': 'invalid_auth', 'errorDetail': 'Missing access credentials. See https://developer.foursquare.com/docs/api/configuration/authentication for details.', 'requestId': '649f3da8972d0922259651cd'}, 'response': {}}
{'meta': {'code': 400, 'errorType': 'invalid_auth', 'errorDetail': 'Missing access credentials. See https://developer.foursquare.com/docs/api/configuration/authentication for details.', 'requestId': '649f3da94de4ff5aaa1af618'}, 'response': {}}
{'meta': {'code': 400, 'errorType': 'invalid_auth', 'errorDetail': 'Missing access credentials. See https://developer.foursquare.com/docs/api/configuration/authentication for details.', 'requestId': '649f3daa93e9d12f46e211b1'}, 'response': {}}


KeyError: 'groups'

Now, we have a dataframe with all the information windowed over for every venue. Lets make it into a binary dataframe.

In [ ]:
usa_venues.groupby('City').count()[0:10]

City Latitude  City Longitude  Venue  Venue Latitude  \
City                                                                  
Abbeville, GA              4               4      4               4   
Aberdeen, ID               2               2      2               2   
Aberdeen, MD              99              99     99              99   
Aberdeen, SD              90              90     90              90   
Aberdeen, WA              61              61     61              61   
Abilene, KS               34              34     34              34   
Abilene, TX              100             100    100             100   
Abingdon, VA              76              76     76              76   
Ada, MN                    4               4      4               4   
Ada, OK                   64              64     64              64   

               Venue Longitude  Venue Category  
City                                            
Abbeville, GA                4               4  
Aberdeen, ID                 2               2  
Aberdeen, MD                99              99  
Aberdeen, SD                90              90  
Aberdeen, WA                61              61  
Abilene, KS                 34              34  
Abilene, TX                100             100  
Abingdon, VA                76              76  
Ada, MN                      4               4  
Ada, OK                     64              64

In [ ]:
print('There are {} uniques categories.'.format(len(usa_venues['Venue Category'].unique())))

There are 611 uniques categories.


In [ ]:
#Get the dummy variables for each venue category
usa_dummies = pd.get_dummies(usa_venues[['Venue Category']], prefix="", prefix_sep="")

#Add name of city to dataframe
usa_dummies['City'] = usa_venues['City']

#Lets make it readable by changing the location of the City column to be the first column of the dataframe
move_column = [usa_dummies.columns[-1]] + list(usa_dummies.columns[:-1])
usa_dummies = usa_dummies[move_column]

print(usa_dummies.shape)
usa_dummies.head()

(146014, 611)


Zoo Exhibit  ATM  Accessories Store  Acupuncturist  Adult Boutique  \
0            0    0                  0              0               0   
1            0    0                  0              0               0   
2            0    0                  0              0               0   
3            0    0                  0              0               0   
4            0    0                  0              0               0   

   Advertising Agency  Afghan Restaurant  African Restaurant  Airport  \
0                   0                  0                   0        0   
1                   0                  0                   0        0   
2                   0                  0                   0        0   
3                   0                  0                   0        0   
4                   0                  0                   0        0   

   Airport Food Court  Airport Lounge  Airport Service  Airport Terminal  \
0                   0               0                0                 0   
1                   0               0                0                 0   
2                   0               0                0                 0   
3                   0               0                0                 0   
4                   0               0                0                 0   

   Airport Tram  Alternative Healer  American Restaurant  Amphitheater  \
0             0                   0                    0             0   
1             0                   0                    0             0   
2             0                   0                    0             0   
3             0                   0                    0             0   
4             0                   0                    0             0   

   Animal Shelter  Antique Shop  Apres Ski Bar  Aquarium  Arcade  \
0               0             0              0         0       0   
1               0             0              0         0       0   
2               0             0              0         0       0   
3               0             0              0         0       0   
4               0             0              0         0       0   

   Arepa Restaurant  Argentinian Restaurant  Art Gallery  Art Museum  \
0                 0                       0            0           0   
1                 0                       0            0           0   
2                 0                       0            0           0   
3                 0                       0            0           0   
4                 0                       0            0           0   

   Arts & Crafts Store  Arts & Entertainment  Asian Restaurant  \
0                    0                     0                 0   
1                    0                     0                 0   
2                    0                     0                 0   
3                    0                     0                 0   
4                    0                     0                 0   

   Assisted Living  Astrologer  Athletics & Sports  Auditorium  \
0                0           0                   0           0   
1                0           0                   0           0   
2                0           0                   0           0   
3                0           0                   0           0   
4                0           0                   0           0   

   Australian Restaurant  Austrian Restaurant  Auto Dealership  Auto Garage  \
0                      0                    0                0            0   
1                      0                    0                0            0   
2                      0                    0                0            0   
3                      0                    0                0            0   
4                      0                    0                0            0   

   Auto Workshop  Automotive Shop  BBQ Joint  Baby Store  Bagel Shop  Bakery  \
0              0                

In [ ]:
usa_venues_grouped = usa_dummies.groupby('City').sum().reset_index()
print(usa_venues_grouped.shape)
usa_venues_grouped.head()

(3928, 611)


City  Zoo Exhibit  ATM  Accessories Store  Acupuncturist  \
0  Abbeville, GA            0    0                  0              0   
1   Aberdeen, ID            0    0                  0              0   
2   Aberdeen, MD            0    0                  0              0   
3   Aberdeen, SD            0    1                  0              0   
4   Aberdeen, WA            0    0                  0              0   

   Adult Boutique  Advertising Agency  Afghan Restaurant  African Restaurant  \
0               0                   0                  0                   0   
1               0                   0                  0                   0   
2               0                   0                  0                   0   
3               0                   0                  0                   0   
4               0                   0                  0                   0   

   Airport  Airport Food Court  Airport Lounge  Airport Service  \
0        0                   0               0                0   
1        0                   0               0                0   
2        0                   0               0                0   
3        0                   0               0                0   
4        0                   0               0                0   

   Airport Terminal  Airport Tram  Alternative Healer  American Restaurant  \
0                 0             0                   0                    0   
1                 0             0                   0                    0   
2                 0             0                   0                    8   
3                 0             0                   0                    8   
4                 0             0                   0                    1   

   Amphitheater  Animal Shelter  Antique Shop  Apres Ski Bar  Aquarium  \
0             0               0             0              0         0   
1             0               0             0              0         0   
2             0               0             0              0         0   
3             0               0             0              0         0   
4             0               0             0              0         0   

   Arcade  Arepa Restaurant  Argentinian Restaurant  Art Gallery  Art Museum  \
0       0                 0                       0            0           0   
1       0                 0                       0            0           0   
2       0                 0                       0            0           0   
3       0                 0                       0            0           0   
4       0                 0                       0            0           0   

   Arts & Crafts Store  Arts & Entertainment  Asian Restaurant  \
0                    0                     0                 0   
1                    0                     0                 0   
2                    1                     0                 0   
3                    0                     0                 1   
4                    0                     0                 0   

   Assisted Living  Astrologer  Athletics & Sports  Auditorium  \
0                0           0                   0           0   
1                0           0                   0           0   
2                0           0                   0           0   
3                0           0                   0           0   
4                0           0                   0           0   

   Australian Restaurant  Austrian Restaurant  Auto Dealership  Auto Garage  \
0                      0                    0                0            0   
1                      0                    0                1            0   
2                      0                    0                0            0   
3                      0                    0                0            0   
4                      0                    0                0            0   

   Auto Workshop  Automotive Sh

In [ ]:
#usa_venues_grouped.to_csv("C:/...")

Great! Now we have the frequency of each category of venue of the top 100 venues in every city.